In [53]:
import pandas as pd
import numpy as np
from plotly.offline import iplot
from plotly.offline import plot
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go

In [54]:
class Logger:
    '''
    Класс для записи задач с длительностямя
    '''
    
    def __init__(self, file=None, log=pd.DataFrame()):
        '''
        Инициализирует структуру для записи задач
        
        Args:
            log(pd.DataFrame): Данные о предыдущем логе, если его нет - создается новый
            
        '''
        if file:
            self.log = pd.read_csv(file, parse_dates=['timestamp'])
            self.state = self.log.iloc[-1]['task']
            self.states = set(self.log.task.unique()) - set(self.log[self.log['status'] == 'end']['status'].unique())
        elif not log.empty:
            self.log = log.copy()
        else:
            self.log = pd.DataFrame(columns=['timestamp', 'task', 'status'])
            self.state = 'idle'
            self.log.loc[self.log.shape[0]] = [pd.Timestamp.now(), self.state, 'begin']
            self.states = set({'idle'})
        
    def begin(self, task):
        '''
        Позволяет начать новую задачу, текущая прерывается при этом
        Args:
            task(str): Имя начинаемой задачи
        '''
        if task not in self.states:
            self.states.add(task)
            self.log.loc[self.log.shape[0]] = [pd.Timestamp.now(), self.state, 'stop']
            self.state = task
            self.log.loc[self.log.shape[0]] = [pd.Timestamp.now(), self.state, 'begin']
        else:
            print('You are already working on this task')
    
    def resume(self, task):
        '''
        Позволяет продолжить существующую задачу, текущая прерывается при этом
        Args:
            task(str): Имя продолжаемой задачи
        '''
        if task in self.states:
            self.log.loc[self.log.shape[0]] = [pd.Timestamp.now(), self.state, 'stop']
            self.state = task
            self.log.loc[self.log.shape[0]] = [pd.Timestamp.now(), self.state, 'resume']
        else:
            print('You can''t resume task that didn''t begin')
            
    def stop(self):
        '''
        Останавливает текущую задачу
        '''
        if self.state != 'idle':
            self.log.loc[self.log.shape[0]] = [pd.Timestamp.now(), self.state, 'stop']
            self.state = 'idle'
            self.log.loc[self.log.shape[0]] = [pd.Timestamp.now(), self.state, 'resume']        
        else:
            print('You can''t end idle state')
    
    def end(self):
        '''
        Завершает текущую задачу
        '''
        if self.state != 'idle':
            self.log.loc[self.log.shape[0]] = [pd.Timestamp.now(), self.state, 'end']
            self.states.remove(self.state)
            self.state = 'idle'
            self.log.loc[self.log.shape[0]] = [pd.Timestamp.now(), self.state, 'resume']        
        else:
            print('You can''t end idle state')
    
    def backup(self):
        self.log.to_csv('log.csv', index=False)
    
    def plot(self, tasks=None):
        '''
        Отрисовка хронометража
        Args:
            tasks(list(str)): Набор задач для рассмотрения, по умолчанию все
        '''
        if not tasks:
            tasks=self.log.task.unique()
        all_states = tasks
        data = []
        shapes = []
        for j, state in enumerate(all_states):
            r = np.random.randint(255)
            g = np.random.randint(255)
            b = np.random.randint(255)
            color = f'rgb({r}, {g}, {b})'
            cur_log = self.log[self.log.task == state]
            size = cur_log.shape[0]
            x = []
            for i in range(0,size//2*2,2):
                x.append(cur_log.iloc[i].timestamp)
                x.append(cur_log.iloc[i+1].timestamp)
                shapes.append({
                      'x0':cur_log.iloc[i].timestamp,
                      'x1':cur_log.iloc[i+1].timestamp,
                      'y0':j-0.2,
                      'y1':j+0.2,
                      'type':'rect',
                      'xref': 'x', 
                      'yref': 'y', 
                      'opacity': 1, 
                      'fillcolor': color
                })
            if size % 2 == 1:
                # Дополнительный интервал, если работа сейчас в процессе выполнения
                x.append(cur_log.iloc[size-1].timestamp)
                x.append(pd.Timestamp.now())
                shapes.append({
                      'x0':cur_log.iloc[size-1].timestamp,
                      'x1':pd.Timestamp.now(),
                      'y0':j-0.2,
                      'y1':j+0.2,
                      'type':'rect',
                      'xref': 'x', 
                      'yref': 'y', 
                      'opacity': 1, 
                      'fillcolor': color
                })
                
            y = [j]*len(x)
            data.append(go.Scatter({
                  'name': state, 
                  'mode': 'markers', 
                  'x': x, 
                  'y': y, 
                  'marker': {
                    'size': 1, 
                    'color': color
                  }, 
                  #'hoverinfo': 'none', 
                  #'showlegend': True
            }))
        layout = {
            'title':'Look',
            'shapes':shapes,
            "hovermode": "closest", 
            "showlegend": False,
            "yaxis": {
                "showgrid": True, 
                "ticktext": all_states, 
                "tickvals": list(range(len(all_states))), 
                "zeroline": False, 
                "autorange": True
            },
        }
        fig={
            'data':data,
            'layout':layout
        }
        iplot(fig)
    
    def analyze(self, tasks=None):
        '''
        Подсчет суммарной длительности по каждой задаче
        
        Args:
            tasks(list(str)): Набор задач для рассмотрения, по умолчанию все
        '''
        if not tasks:
            tasks=self.log.task.unique()
        all_states = tasks
        rows = []
        for j, state in enumerate(all_states):
            cur_log = self.log[self.log.task == state].copy().reset_index().drop(columns='index')
            if cur_log.shape[0] % 2 == 1:
                cur_log.loc[cur_log.shape[0]] = [pd.Timestamp.now(), state, 'end']   
            upper_bound = np.array(cur_log['timestamp'].iloc[1::2])
            lower_bound = np.array(cur_log['timestamp'].iloc[0::2])
            row = [state, (upper_bound - lower_bound).sum()]
            rows.append(row)
        return pd.DataFrame(rows, columns=['task', 'duration'])
        

# Создание

In [65]:
logger = Logger('log.csv')

# Операции

In [70]:
logger.begin('Эксперименты')

In [59]:
logger.stop()

In [50]:
logger.resume('Ori')

In [71]:
logger.end()

In [64]:
logger.backup()

# Обзор

In [73]:
logger.state

'idle'

In [67]:
logger.states 

{'Ori', 'idle'}

In [74]:
logger.plot()

In [72]:
logger.analyze()

,task,duration
0,idle,14:08:09.475618
1,Ori,01:34:15.520458
2,Эксперименты,00:44:55.507455


In [58]:
logger.log

,timestamp,task,status
0,2020-03-20 19:18:16.718814,idle,begin
1,2020-03-20 19:18:17.699155,idle,stop
2,2020-03-20 19:18:17.702957,Ori,begin
3,2020-03-20 19:18:17.986907,Ori,stop
4,2020-03-20 19:18:17.990761,idle,resume
5,2020-03-20 19:24:06.721636,idle,stop
6,2020-03-20 19:24:06.724954,Ori,resume


In [52]:
logger.backup()

In [811]:
log_backup = logger.log.copy()

In [812]:
log_backup

,timestamp,task,status
0,2020-03-13 11:25:11.559632,idle,begin
1,2020-03-13 11:25:38.711118,idle,stop
2,2020-03-13 11:25:38.715329,Описание идей для прогноза LTV,begin
3,2020-03-13 11:30:58.734220,Описание идей для прогноза LTV,end
4,2020-03-13 11:30:58.754867,idle,resume
5,2020-03-13 11:47:24.974880,idle,stop
6,2020-03-13 11:47:25.002364,Подготовка презентации к спецсему,begin
7,2020-03-13 12:18:26.082281,Подготовка презентации к спецсему,stop
8,2020-03-13 12:18:26.194548,idle,resume
9,2020-03-13 12:20:49.824232,idle,stop
